In [31]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:16pt;}
div.text_cell_render.rendered_html{font-size:16pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:16pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
table.dataframe{font-size:16px;}
</style>
"""))

# <b><font size="7" color="red">ch15. 데이터베이스 연동</font></b>
# 1장 SQLite 데이터 베이스 연결
- SQLite 데이터 베이스는 별도의 DBMS 없이 SQL문 이용해서 DB엑세스할 수 있도록 만든 간단한 디스크 기반 DB
- C라이브러리
- SQLite는 프로토타입 생성시 사용
- 프로젝트 : 분석 -> 설계 -> 구현 -> 테스트 -> 고객에게 배포 -> 유지보수
                프로토타입(시제품) 완제품(Oracle, MySQL, Maria, ..)
- DB browser for SQLite
## 1.1 SQLite browser 설치 및 SQLite 패키지 load

In [16]:
import sqlite3
sqlite3.sqlite_version # 버전

'3.40.1'

In [15]:
import pandas as pd
pd.__version__

'1.5.3'

## 1.2 데이터 베이스 연결
```
(1) 데이터베이스 연결
(2) SQL 전송 객체 생성(cursor)
(3) SQL 전송 & 결과 받기(cursor.execute()이용)
(4) cursor 해제 & 데이터베이스 연결객체 해제
```

In [4]:
# (1) DB 연결
conn = sqlite3.connect('data/ch15_example.db')
conn

In [10]:
# (2) SQL 전송 객체 생성(cursor 객체)
cursor = conn.cursor()
cursor

In [13]:
cursor.execute('''
    CREATE TABLE MEMBER(
    NAME TEXT,
    AGE INT,
    EMAIL TEXT
    )
    ''')

In [12]:
cursor.execute('DROP TABLE MEMBER')

In [14]:
cursor.execute('INSERT INTO MEMBER VALUES (\'홍길동\', 20, \'H@H.COM\')')
print('수행결과 행수 :', cursor.rowcount)

수행결과 행수 : 1


In [17]:
sql = "INSERT INTO MEMBER VALUES ('김길동', 30, 'K@K.COM')"
cursor.execute(sql)
print('수행 결과 행수 :', cursor.rowcount)
cursor.execute("INSERT INTO MEMBER VALUES ('이길동', 25, 'l@l.com')")
print('수행 결과 행수 :', cursor.rowcount)

수행 결과 행수 : 1
수행 결과 행수 : 1


In [18]:
conn.commit() # (반)conn.rollback()

In [19]:
# select 전송 결과 : cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER")

In [20]:
# insert, update, delete문 실행 결과 : cursor.rowcount
# select문 실행 결과 :
        ## fetchall() : 결과를 모두 받을 때(튜플 list)
        ## fetchone() : 결과를 한행씩 받을 때(튜플)
        ## fetchmany(n) : 결과를 n행 받을 때(튜플 list)
print(cursor.fetchall())

[('홍길동', 20, 'H@H.COM'), ('김길동', 30, 'K@K.COM'), ('이길동', 25, 'l@l.com')]


In [21]:
print(cursor.fetchall()) # 한번 소요된 cursor 객체는 다시 fetch될 수 없음

[]


In [24]:
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members

[('홍길동', 20, 'H@H.COM'), ('이길동', 25, 'l@l.com'), ('김길동', 30, 'K@K.COM')]

In [25]:
for member in members:
    print(member)

('홍길동', 20, 'H@H.COM')
('이길동', 25, 'l@l.com')
('김길동', 30, 'K@K.COM')


In [26]:
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
memebers=[]
while True:
    member = cursor.fetchone()
    if member is None:
        print('데이터 끝')
        break
    members.append(member)

데이터 끝


In [27]:
members

[('홍길동', 20, 'H@H.COM'),
 ('이길동', 25, 'l@l.com'),
 ('김길동', 30, 'K@K.COM'),
 ('홍길동', 20, 'H@H.COM'),
 ('이길동', 25, 'l@l.com'),
 ('김길동', 30, 'K@K.COM')]

In [28]:
# 최상위 n행 읽기
cursor.execute("SELECT * FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(2):
    print(member)

('홍길동', 20, 'H@H.COM')
('이길동', 25, 'l@l.com')


In [29]:
cursor.description

(('NAME', None, None, None, None, None, None),
 ('AGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [33]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age  = age
        self.email = email
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)
def to_member(*row): # 튜플매개변수를 받아 Member형 객체를 return
    return Member(row[0], row[1], row[2])

In [34]:
member = to_member('홍길동', 20, 'h@h.com')
print(member)

홍길동	20	h@h.com


In [36]:
cursor.execute('SELECT * FROM MEMBER')
member_list = [] # sql문 수행한 결과를 담을 객체 list
members = cursor.fetchall() # 튜플 list
for member in members:
    member_list.append(to_member(*member))

In [37]:
for member in member_list:
    print(member)

홍길동	20	H@H.COM
김길동	30	K@K.COM
이길동	25	l@l.com


In [38]:
# (4) 연결 해제
cursor.close()
conn.close()

## 1,3 SQL 구문에 파라미터 사용하기
- qmark(DB에 따라 불가한 경우 있음)
- named(추천)

In [39]:
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','김길동')")
cursor.fetchall()

[('홍길동', 20, 'H@H.COM'), ('김길동', 30, 'K@K.COM')]

In [40]:
# 파라미터 사용하기 : qmark 방법 
name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
# cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('"+name1+"','"+name2+"')")
# cursor.execute(f"SELECT * FROM MEMBER WHERE NAME IN ('{name1}','{name2}')")
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?, ?)", (name1, name2))
cursor.fetchall()

검색할 이름1?ㅌㅌ
검색할 이름2?ㄷㄷ


[]

In [41]:
# 파라미터 사용하기 : named 방법 
name1 = input('검색할 이름1?')
name2 = input('검색할 이름2?')
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)", {'name1':name1,'name2':name2})
cursor.fetchall()

검색할 이름1?홍길동
검색할 이름2?신길동


[('홍길동', 20, 'H@H.COM')]

In [42]:
# Member 테이블에 입력(사용자로부터 이름,나이,메일을 받아 insert)
try :
    name = input('입력할 이름 :')
    age = int(input('입력할 나이 :'))
except:
    print('유효하지않는 나이입니다. 18세로 입력됩니다')
    age = 18
finally:
    email = input('입력할 메일은 :')
inputdata = {'name':name, 'age':age, 'email':email} # named 방식에서 사용
inputdata2 = (name, age, email) # qmark방식에서 사용
sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
cursor.execute(sql, inputdata)
conn.commit()
if cursor.rowcount:
    print('저장완료')

입력할 이름 :타로
입력할 나이 :39
입력할 메일은 :t@t.com
저장완료


In [51]:
while True:
    try :
        name = input('입력할 이름(종료는 0) :')
        if name == '0':
            break
        age = int(input('입력할 나이 :'))
    except:
        print('유효하지않는 나이입니다. 18세로 입력됩니다')
        age = 18
    email = input('입력할 메일은 :')
    newMember = Member(name, age, email)
    sql = "INSERT INTO MEMBER VALUES (:name, :age, :email)"
    cursor.execute(sql, newMember.__dict__)
    if cursor.rowcount==1:
        print('입력 성공')
conn.commit()

입력할 이름(종료는 0) :0


In [58]:
cursor.close()
conn.close()

# 2장 오라클 데이터 베이스 연결
- pip install cx_oracle (oracle 11g까지)
- pip install oracledb  (oracle 12부터)

In [61]:
import cx_Oracle
cx_Oracle.__version__

'8.3.0'

In [62]:
# conn 얻어오는 방법1
conn = cx_Oracle.connect("scott", "tiger", "localhost:1521/xe")
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO", ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [63]:
# conn 얻어오는 방법2
import oracledb
oracledb.init_oracle_client()
# conn = oracledb.connect("scott/tiger@localhost:1521/xe")
conn = oracledb.connect(
    user="scott",
    password="tiger",
    host="localhost",
    port=1521,
    sid="xe"
)
cursor = conn.cursor()
sql = 'SELECT EMPNO "NO", ENAME, JOB, MGR, HIREDATE, SAL, COMM, DEPTNO FROM EMP'
cursor.execute(sql)
emp = cursor.fetchall()
print(emp)

[(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20), (7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30), (7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30), (7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20), (7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30), (7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30), (7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10), (7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20), (7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10), (7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30), (7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20), (7900, 

In [64]:
import pandas as pd
emp_df = pd.DataFrame(emp)
emp_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [65]:
# select문을 수행한 필드 정보(NO, ENAME, JOB ~)
cursor.description

[('NO', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, False),
 ('ENAME', <DbType DB_TYPE_VARCHAR>, 10, 10, None, None, True),
 ('JOB', <DbType DB_TYPE_VARCHAR>, 9, 9, None, None, True),
 ('MGR', <DbType DB_TYPE_NUMBER>, 5, None, 4, 0, True),
 ('HIREDATE', <DbType DB_TYPE_DATE>, 23, None, None, None, True),
 ('SAL', <DbType DB_TYPE_NUMBER>, 11, None, 7, 2, True),
 ('COMM', <DbType DB_TYPE_NUMBER>, 11, None, 7, 2, True),
 ('DEPTNO', <DbType DB_TYPE_NUMBER>, 3, None, 2, 0, True)]

In [66]:
[description[0] for description in cursor.description]

['NO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [67]:
emp_df.columns = [description[0] for description in cursor.description]
emp_df.head()

,NO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [68]:
# 검색할 이름을 사용자에게 받아 해당 내용을 출력
ename = input('검색할 이름은?').upper()
# print(ename)
sql = "SELECT * FROM EMP WHERE ENAME = :ename"
cursor.execute(sql, {'ename':ename})
emp = cursor.fetchall()
print(emp)

검색할 이름은?scott
[(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)]


In [73]:
fieldnames = [descrip[0] for descrip in cursor.description]
fieldnames

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [74]:
for fieldname, data in zip(fieldnames, emp[0]):
    print("{}:{}".format(fieldname, data))

EMPNO:7788
ENAME:SCOTT
JOB:ANALYST
MGR:7566
HIREDATE:1982-12-09 00:00:00
SAL:3000.0
COMM:None
DEPTNO:20


In [75]:
cursor.close()
conn.close()

# 3장 MySQL 데이터베이스 연결
- pip install mysql-connector-python (공식connector)
- pip install PyMySQL (경량)

In [76]:
%pip install PyMySQL

     ---------------------------------------- 45.3/45.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [80]:
import pymysql
conn = pymysql.connect(
    host = "localhost",
    user = "root",
    password = "mysql",
    db = "devdb",
    charset="utf8mb4" # 한글이 있다면 입력
)
cursor = conn.cursor()
sql ="SELECT * FROM PERSONAL"
cursor.execute(sql)
personal = cursor.fetchall()
fieldnames = [descript[0] for descript in cursor.description]
pd.DataFrame(personal, columns=fieldnames)

,pno,pname,job,manager,startdate,pay,bonus,dno
0,1001,bill,president,NaN,1989-01-10,7000,NaN,10
1,1111,smith,manager,1001.0,1990-12-17,1000,NaN,10
2,1112,ally,salesman,1116.0,1991-02-20,1600,500.0,30
3,1113,word,salesman,1116.0,1992-02-24,1450,300.0,30
4,1114,james,manager,1001.0,1990-04-12,3975,NaN,20
5,1116,johnson,manager,1001.0,1991-05-01,3550,NaN,30
6,1118,martin,analyst,1111.0,1991-09-09,3450,NaN,10
7,1121,kim,clerk,1114.0,1990-12-08,4000,NaN,20
8,1123,lee,salesman,1116.0,1991-09-23,1200,0.0,30
9,1226,park,analyst,1111.0,1990-01-03,2500,NaN,10


In [81]:
cursor.close()
conn.close()

# 4장 연습문제
## 실습형
- 회원가입 | 전체조회 | 이름찾기 | 메일삭제 | csv보내기 | 종료
### 0. 처음실행

In [113]:
def load_cursor():
    global conn
    import oracledb
    import pandas as pd
    oracledb.init_oracle_client()
    conn = oracledb.connect("scott/tiger@localhost")
load_cursor()

### 1. 입력

In [93]:
def fn1_insert_member_info():
    '입력받은 데이터를 member테이블에 입력'
    cursor = conn.cursor()
    name = input('이름 :')
    tel = int(input('전화 :'))
    email = input('이메일 :')
    try:
        age = int(input('나이 :'))
        if age > 200:
            age = 200
        elif age<0:
            age=0
    except:
        print('유효하지 않은 나이입니다. 0으로 초기화합니다')
        age = 0
    try:
        grade = int(input('등급 :'))
        if grade<1:
            grade =1
        elif grade>5:
            grade = 5
    except:
        print('유효하지 않은 등급입니다 등급은 1로 초기화합니다')
        grade = 1
    etc = input('기타정보 :')
    sql = "INSERT INTO MEMBER VALUES (:name, :tel, :email, :age, :grade, :etc)"
    inputdata = {'name':name, 'tel':tel, 'email':email, 'age':age, 'grade':grade, 'etc':etc}
    cursor.execute(sql, inputdata)
    conn.commit()
    if cursor.rowcount==1:
        print(name, '님 가입 완료')
    cursor.close()

In [94]:
fn1_insert_member_info()

이름 :ㅎㅎ
전화 :101
이메일 :er@er.com
나이 :22
등급 :3
기타정보 :dd
ㅎㅎ 님 가입 완료


In [136]:
def fn2_print_members():
    'member 테이블의 내용을 데이터프레임으로 display'
    sql = "SELECT * FROM MEMBER"
    cursor = conn.cursor()
    cursor.execute(sql)
    personal = cursor.fetchall()
    fieldnames = [descript[0] for descript in cursor.description]
    display(pd.DataFrame(personal, columns=fieldnames))
    cursor.close()

In [121]:
fn2_print_members()

,NAME,TEL,EMAIL,AGE,GRADE,ETC
0,김길동,1001,ㅁㅁ@ㅁㄴㅇ.채ㅡ,11,1,ㅇㅇ
1,ㅎㅎ,101,er@er.com,22,3,dd


In [135]:
def fn3_find_members():
    cursor = conn.cursor()
    name = input('검색할 이름은?').upper()
    sql = "SELECT * FROM MEMBER WHERE NAME = :name"
    cursor.execute(sql, {'name':name})
    member = cursor.fetchall()
    print(member)
    cursor.close()

In [134]:
fn3_find_members()

검색할 이름은?ㅎㅎ
[]


In [144]:
def fn4_del_email():
    cursor = conn.cursor()
    email = input('삭제할 이메일은?')
    sql = "DELETE FROM MEMBER WHERE EMAIL = :email"
    cursor.execute(sql, {'email':email})
    conn.commit()
    print('이메일이 삭제되었습니다')
    cursor.close()

In [145]:
fn4_del_email()

삭제할 이메일은?er@er.com
이메일이 삭제되었습니다
